In [28]:
import sqlalchemy as sa
import pandas as pd
from pyvis.network import Network
import urllib
from itertools import product,combinations

In [29]:
params = urllib.parse.quote_plus('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=localhost;'
                      'Database=InteractDb_rc2;'
                      'Trusted_Connection=yes;')

engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))
cnxn = engine.connect()

In [30]:
net = Network(directed=True, notebook=True, cdn_resources='remote')
net.set_edge_smooth('dynamic')
options = '''
const options = {
  "physics": {
    "barnesHut": {
      "gravitationalConstant": -16150,
      "centralGravity": 0,
      "springLength": 155
    },
    "minVelocity": 0.75
  }
}
'''
net.set_options(options)

# Utility functions

In [31]:
def get_drugbank_ids(syn):
    qry = 'SELECT * from drug_nodes WHERE synonyms LIKE \'%' +  syn + '%\''
    df = pd.read_sql_query(qry, cnxn)
    ids = None
    if len(df) > 0:
        ids = df['drugbank_id'].to_list()
    return ids


# Search space seeds

In [32]:
x = "cyclosporine"
y = "tacrolimus"

In [33]:
x_ids, y_ids = get_drugbank_ids(x), get_drugbank_ids(y)
search_set = x_ids + y_ids
search_set = list(set(search_set))
search_space = list(product(x_ids, y_ids))
search_space

[('DB00091', 'DB00864')]

In [34]:
drug_group_id = 1
gene_group_id = 2
disease_group_id = 3

# Direct drug interactions

In [35]:
dfs = []

for s in search_space:
    x_, y_ = s
    qry = '''\
    SELECT Drug1.drugbank_id AS src_id, Drug1.name AS 'Source Drug', Drug2.drugbank_id as dst_id, Drug2.name AS 'Interacting Drug', [dbo].[drug_drug_edges].[desc] AS 'Effect of Interaction'
    FROM drug_nodes Drug1, drug_drug_edges, drug_nodes Drug2
    WHERE MATCH(Drug1-(drug_drug_edges)->Drug2)
    AND Drug1.drugbank_id = '{x}'
    AND Drug2.drugbank_id = '{y}'\
    '''.format(x=x_, y=y_)
    df = pd.read_sql_query(qry, cnxn)
    dfs.append(df)

dfs = pd.concat(dfs)


In [36]:
dfs

,src_id,Source Drug,dst_id,Interacting Drug,Effect of Interaction
0,DB00091,Cyclosporine,DB00864,Tacrolimus,Tacrolimus may increase the nephrotoxic activi...


In [37]:
for idx, row in dfs.iterrows():
    x, y = row["Source Drug"], row["Interacting Drug"]
    x_id, y_id = row["src_id"], row["dst_id"]
    net.add_node(x_id, title=x_id, label=x, group=drug_group_id)
    net.add_node(y_id, title=y_id, label=y, group=drug_group_id)
    net.add_edge(x_id,y_id, title=str(row["Effect of Interaction"]))


# Drug gene interactions

In [38]:
search_set

['DB00864', 'DB00091']

In [39]:
dfs = []
for item in search_set:
    qry = '''\
    SELECT Drug1.drugbank_id, Drug1.name AS [Drug 1], Genes.gene_names, Genes.ensembl_id, drug_gene_edges.action
    FROM drug_nodes Drug1, drug_gene_edges, gene_nodes Genes
    WHERE MATCH(Drug1-(drug_gene_edges)->Genes)
    AND Drug1.drugbank_id = '{x}'\
    '''.format(x=item)
    df = pd.read_sql_query(qry, cnxn)
    dfs.append(df)

dfs = pd.concat(dfs)

In [40]:
dfs

,drugbank_id,Drug 1,gene_names,ensembl_id,action
0,DB00864,Tacrolimus,FKBP1A,ENSP00000383003,inhibitor
1,DB00864,Tacrolimus,FKBP1,ENSP00000383003,inhibitor
2,DB00864,Tacrolimus,FKBP12,ENSP00000383003,inhibitor
0,DB00091,Cyclosporine,CAMLG,ENSP00000297156,binder
1,DB00091,Cyclosporine,CAML,ENSP00000297156,binder
2,DB00091,Cyclosporine,GET2,ENSP00000297156,binder
3,DB00091,Cyclosporine,PPP3R2,ENSP00000363939,inhibitor
4,DB00091,Cyclosporine,CBLP,ENSP00000363939,inhibitor
5,DB00091,Cyclosporine,PPP3RL,ENSP00000363939,inhibitor
6,DB00091,Cyclosporine,PPIA,ENSP00000419425,inhibitor


In [41]:
# dfg = pd.DataFrame(dfs.groupby('ensembl_id')['gene_names'].apply(', '.join))
# dfg = pd.DataFrame(dfg.to_records())
# dfs = pd.merge(left=dfg, right=dfs[["drugbank_id", "Drug 1", "ensembl_id", "action"]], on="ensembl_id", how="inner")
# dfs = dfs.drop_duplicates()

In [42]:
for idx, row in dfs.iterrows():
    x, y, z = row["Drug 1"], row["gene_names"], row["action"]
    x_id, y_id = row["drugbank_id"], row["gene_names"]
    net.add_node(x_id, title=x_id, label=x, group=drug_group_id)
    net.add_node(y_id, title=y_id, label=y, group=gene_group_id)
    net.add_edge(x_id,y_id, title=str(z))

# Gene Gene interactions

In [43]:
gene_search_space_ = list(set(dfs["ensembl_id"].to_list()))
gene_search_space = list(combinations(gene_search_space_, 2))
gene_search_space

[('ENSP00000225174', 'ENSP00000419425'),
 ('ENSP00000225174', 'ENSP00000363939'),
 ('ENSP00000225174', 'ENSP00000383003'),
 ('ENSP00000225174', 'ENSP00000297156'),
 ('ENSP00000419425', 'ENSP00000363939'),
 ('ENSP00000419425', 'ENSP00000383003'),
 ('ENSP00000419425', 'ENSP00000297156'),
 ('ENSP00000363939', 'ENSP00000383003'),
 ('ENSP00000363939', 'ENSP00000297156'),
 ('ENSP00000383003', 'ENSP00000297156')]

In [44]:
dfs = []

for g in gene_search_space:
    x_, y_ = g
    qry = '''\
    SELECT Gene1.gene_names AS 'Gene 1', Gene2.gene_names AS 'Gene 2'
    FROM gene_nodes Gene1, gene_gene_edges, gene_nodes Gene2
    WHERE MATCH(Gene1-(gene_gene_edges)->Gene2)
    AND Gene1.ensembl_id = '{x}'
    AND Gene2.ensembl_id = '{y}'\
    '''.format(x=x_, y=y_)
    df = pd.read_sql_query(qry, cnxn)
    dfs.append(df)

dfs = pd.concat(dfs)


In [45]:
dfs

,Gene 1,Gene 2
0,CYP3,CYPA
0,CYP3,FKBP12
0,CYPA,FKBP12
0,PPP3RL,FKBP12
0,PPP3RL,GET2


In [46]:
for idx, row in dfs.iterrows():
    x, y = row["Gene 1"], row["Gene 2"]
    net.add_edge(x,y)

# Disease associations

## with Genes

In [47]:
gene_search_space_

['ENSP00000225174',
 'ENSP00000419425',
 'ENSP00000363939',
 'ENSP00000383003',
 'ENSP00000297156']

In [55]:
dfs = []

for g in gene_search_space_:
    qry = '''\
    SELECT Disease.name AS [Disease Name], Disease.vocabulary, Genes.gene_names as [Associated Genes], disease_gene_edges.sentence
    FROM disease_nodes Disease, disease_gene_edges, gene_nodes Genes
    WHERE MATCH (Genes-(disease_gene_edges)->Disease)
    AND Genes.ensembl_id = '{x}'\
        '''.format(x=g)
    df = pd.read_sql_query(qry, cnxn)
    dfs.append(df)

dfs = pd.concat(dfs)

In [56]:
dfs = dfs.drop_duplicates()
dfs

,Disease Name,vocabulary,Associated Genes,sentence
0,Malignant tumor of colon,NCI,CYP3,This study aims to assess the effects of GAD o...
1,Cardiovascular Diseases,NCI,CYP3,A greater understanding of the physiologic fun...
2,Coronary Arteriosclerosis,NCI,CYP3,Mitochondrial permeability transition in cardi...
3,Diabetes,MONDO,CYP3,We tested the hypothesis of whether NMDA-R1 me...
4,Diabetes,MONDO,CYP3,Brains from mice with STZ-induced diabetes rev...
...,...,...,...,...
8,Breast Carcinoma,ORDO,GET2,These results suggest a new role for CAML in b...
9,Adult Lymphoma,NCI,GET2,"A proliferation-inducing ligand (APRIL), as we..."
10,Childhood Leukemia,NCI,GET2,Breast cancer cell lines were found to express...
11,Childhood Lymphoma,NCI,GET2,"A proliferation-inducing ligand (APRIL), as we..."


In [50]:
for idx, row in dfs.iterrows():
    dname = row["Disease Name"]
    gene = row["Associated Genes"]
    net.add_node(dname, title=dname, label=dname, group=disease_group_id)
    net.add_edge(dname, gene, title=str(row["sentence"]))

## with drugs

In [57]:
dfs = []

for d in search_set:
    qry = '''\
    SELECT Disease.vocab_name AS [Disease Name], Disease.MeSH_heading, Drugs.name as [Associated Drugs]
    FROM disease_nodes Disease, drug_disease_edges, drug_nodes Drugs
    WHERE MATCH (Drugs-(drug_disease_edges)->Disease)
    AND Drugs.drugbank_id = '{x}'\
        '''.format(x=d)
    df = pd.read_sql_query(qry, cnxn)
    dfs.append(df)

dfs = pd.concat(dfs)
dfs


,Disease Name,MeSH_heading,Associated Drugs
0,pustulosis of palm and sole,Skin and Connective Tissue Diseases,Tacrolimus
1,vitiligo,Skin and Connective Tissue Diseases,Tacrolimus
2,psoriasis,Skin and Connective Tissue Diseases,Tacrolimus
3,Vitiligo,Skin and Connective Tissue Diseases,Tacrolimus
4,psoriasis,Skin and Connective Tissue Diseases,Tacrolimus
...,...,...,...
1208,"Cystitis, Chronic Interstitial",Female Urogenital Diseases and Pregnancy Compl...,Cyclosporine
1209,Painful Bladder Syndrome,Male Urogenital Diseases,Cyclosporine
1210,Painful Bladder Syndrome,Female Urogenital Diseases and Pregnancy Compl...,Cyclosporine
1211,Interstitial cystitis,Male Urogenital Diseases,Cyclosporine


In [52]:
for e in net.edges:
  e['arrows'] = {'to': {'scaleFactor': 0.5}}
#net.show_buttons(filter_=['physics'])
net.show("demo.html")

demo.html
